## Import Dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [ ]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import pymysql
pymysql.install_as_MySQLdb()

In [ ]:
engine = create_engine(f"mysql://{root}:{password}@{localhost}/{database}")
inspector = inspect(engine)

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
session = Session(engine)

In [ ]:
Base = declarative_base()

### Extract CSVs into DataFrames

In [3]:
data = pd.read_csv('./athlete_events.csv')
country_info = pd.read_csv('./noc_regions.csv')
olympic_df = pd.merge(data, country_info, how="left", on = "NOC")
olympic_df.head()

ID                      Name Sex   Age  Height  Weight            Team  \
0   1                 A Dijiang   M  24.0   180.0    80.0           China   
1   2                  A Lamusi   M  23.0   170.0    60.0           China   
2   3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN         Denmark   
3   4      Edgar Lindenau Aabye   M  34.0     NaN     NaN  Denmark/Sweden   
4   5  Christine Jacoba Aaftink   F  21.0   185.0    82.0     Netherlands   

   NOC        Games  Year  Season       City          Sport  \
0  CHN  1992 Summer  1992  Summer  Barcelona     Basketball   
1  CHN  2012 Summer  2012  Summer     London           Judo   
2  DEN  1920 Summer  1920  Summer  Antwerpen       Football   
3  DEN  1900 Summer  1900  Summer      Paris     Tug-Of-War   
4  NED  1988 Winter  1988  Winter    Calgary  Speed Skating   

                              Event Medal       region notes  
0       Basketball Men's Basketball   NaN        China   NaN  
1      Judo Men's Extra-Lightweight   NaN        China   NaN  
2           Football Men's Football   NaN      Denmark   NaN  
3       Tug-Of-War Men's Tug-Of-War  Gold      Denmark   NaN  
4  Speed Skating Women's 500 metres   NaN  Netherlands   NaN

In [4]:
olympic_info = olympic_df.drop(columns=['Name', 'Age', "Height", "Weight", "Games", "Season", "City", "Sport", "Event", "ID","notes"])

In [7]:
olympic_df = olympic_info.dropna()
olympic_df.reset_index(drop=True)

Sex            Team  NOC  Year   Medal       region
0       M  Denmark/Sweden  DEN  1900    Gold      Denmark
1       M         Finland  FIN  1920  Bronze      Finland
2       M         Finland  FIN  1920  Bronze      Finland
3       M         Finland  FIN  2014  Bronze      Finland
4       M         Finland  FIN  1948  Bronze      Finland
5       M         Finland  FIN  1948    Gold      Finland
6       M         Finland  FIN  1948    Gold      Finland
7       M         Finland  FIN  1948    Gold      Finland
8       M         Finland  FIN  1952  Bronze      Finland
9       M          Norway  NOR  1992    Gold       Norway
10      M          Norway  NOR  1992  Bronze       Norway
11      M          Norway  NOR  1994  Silver       Norway
12      M          Norway  NOR  1994  Bronze       Norway
13      M          Norway  NOR  1994  Silver       Norway
14      M          Norway  NOR  2002    Gold       Norway
15      M          Norway  NOR  2002    Gold       Norway
16      M          Norway  NOR  2006    Gold       Norway
17      F          Norway  NOR  2008    Gold       Norway
18      M          Norway  NOR  1920  Silver       Norway
19      F     Netherlands  NED  1988  Bronze  Netherlands
20      M     Netherlands  NED  1996  Silver  Netherlands
21      F          Norway  NOR  1996  Bronze       Norway
22      M          Norway  NOR  1920  Silver       Norway
23      M          Norway  NOR  1952  Bronze       Norway
24      M          Norway  NOR  1960    Gold       Norway
25      M          Taifun  NOR  1912    Gold       Norway
26      M          France  FRA  1956    Gold       France
27      M           Italy  ITA  2016  Bronze        Italy
28      M           Spain  ESP  2008  Bronze        Spain
29      F      Azerbaijan  AZE  2016  Bronze   Azerbaijan
...    ..             ...  ...   ...     ...          ...
39744   M     Switzerland  SUI  1896  Silver  Switzerland
39745   M     Switzerland  SUI  1896  Silver  Switzerland
39746   M     Switzerland  SUI  1896    Gold  Switzerland
39747   M         Belarus  BLR  2004  Silver      Belarus
39748   F          Russia  RUS  2008    Gold       Russia
39749   F          Russia  RUS  2012  Silver       Russia
39750   M      Yugoslavia  YUG  1956  Silver       Serbia
39751   M        Slovenia  SLO  1992  Bronze     Slovenia
39752   F         Belarus  BLR  1996  Bronze      Belarus
39753   F         Belarus  BLR  2000    Gold      Belarus
39754   F    Unified Team  EUN  1992  Bronze       Russia
39755   M         Georgia  GEO  2004    Gold      Georgia
39756   M    Soviet Union  URS  1976  Bronze       Russia
39757   F          Russia  RUS  2008  Bronze       Russia
39758   F         Germany  GER  2004    Gold      Germany
39759   M         Austria  AUT  1924  Silver      Austria
39760   M     Netherlands  NED  2004  Silver  Netherlands
39761   M     Netherlands  NED  1992  Silver  Netherlands
39762   M     Netherlands  NED  1996    Gold  Netherlands
39763   M         Germany  GER  2016  Bronze      Germany
39764   F     Netherlands  NED  1996  Bronze  Netherlands
39765   M          Poland  POL  1980  Silver       Poland
39766   M     Netherlands  NED  1992  Bronze  Netherlands
39767   M     Netherlands  NED  1996    Gold  Netherlands
39768   F    Soviet Union  URS  1952    Gold       Russia
39769   F    Soviet Union  URS  1956  Silver       Russia
39770   F    Soviet Union  URS  1964  Bronze       Russia
39771   M          Poland  POL  1980  Bronze       Poland
39772   F          Russia  RUS  2000  Bronze       Russia
39773   F          Russia  RUS  2004  Silver       Russia

[39774 rows x 6 columns]

In [12]:
# It looks like some of the teams aren't countries, so this is another difficulty to contend with
# Changed the grup by to "region" instead of team to get country name

#olympic_df.groupby("region").count().head(15)

In [10]:
# Total team names is more than total countries in country GDP dataset, where 176 countries are represented
# Number of countries is down to 136 
number_of_countries = olympic_df["region"].nunique()
number_of_countries

136

In [14]:
olympic_df["medal_won"] = (olympic_df["Medal"] != "NaN")
olympic_df.head()

C:\Users\cynth\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Sex            Team  NOC  Year   Medal   region  medal_won
3    M  Denmark/Sweden  DEN  1900    Gold  Denmark       True
37   M         Finland  FIN  1920  Bronze  Finland       True
38   M         Finland  FIN  1920  Bronze  Finland       True
40   M         Finland  FIN  2014  Bronze  Finland       True
41   M         Finland  FIN  1948  Bronze  Finland       True

In [15]:
infile = pd.read_csv("Country_GDP.csv")
country_GDP = pd.DataFrame(infile)

In [19]:
country_GDP = country_GDP.sort_values(by = "Country").reset_index()
country_GDP.head()

level_0  index            Country 1913    1950    1973    1990    1998
0        0      0        Afghanistan  NaN   5 255   9 181   8 861  12 744
1        1      1            Albania  NaN   1 228   5 219   8 125   7 999
2        2      2            Algeria  NaN  12 136  35 814  73 934  81 948
3        3      3             Angola  NaN   4 331  10 784   7 207   7 029
4        4      4  Antigua & Barbuda  NaN      82     328     413     510

In [20]:
country_GDP = country_GDP.drop(columns = ['level_0', 'index', 'Unnamed: 1'])

KeyError: "['Unnamed: 1'] not found in axis"

In [21]:
infile = pd.read_csv("Country_population.csv")
country_pop = pd.DataFrame(infile)

In [22]:
country_pop.head()

Country   1913   1950    1973    1990    1998
0        Afghanistan  5 730  8 150  13 421  14 767  24 792
1            Albania    898  1 227   2 318   3 273   3 331
2            Algeria    NaN  8 893  15 198  25 352  30 481
3             Angola    NaN  4 118   6 028   8 430  10 865
4  Antigua & Barbuda    NaN     46      68      63      64

In [23]:
country_pop = country_pop.rename(columns = {'Unnamed: 0': "Country"}).sort_values(by = 'Country').reset_index()

In [24]:
country_pop.head()

index            Country   1913   1950    1973    1990    1998
0      0        Afghanistan  5 730  8 150  13 421  14 767  24 792
1      1            Albania    898  1 227   2 318   3 273   3 331
2      2            Algeria    NaN  8 893  15 198  25 352  30 481
3      3             Angola    NaN  4 118   6 028   8 430  10 865
4      4  Antigua & Barbuda    NaN     46      68      63      64

In [25]:
country_pop = country_pop.drop(columns = ['Unnamed: 1', 'index'])
country_pop.head()

KeyError: "['Unnamed: 1'] not found in axis"

In [26]:
country_pop = country_pop.sort_values(by = 'Country').reset_index()

In [27]:
country_pop.head()

level_0  index            Country   1913   1950    1973    1990    1998
0        0      0        Afghanistan  5 730  8 150  13 421  14 767  24 792
1        1      1            Albania    898  1 227   2 318   3 273   3 331
2        2      2            Algeria    NaN  8 893  15 198  25 352  30 481
3        3      3             Angola    NaN  4 118   6 028   8 430  10 865
4        4      4  Antigua & Barbuda    NaN     46      68      63      64

In [28]:
country_pop.to_csv("Country_population.csv", index = False)